**因 Jupter 对多进程支持不完善，因此分布式实验的代码运行均需要通过脚本执行！！！！！！**

## 1.背景介绍

本实验主要研究 PyTorch 分布式训练框架下的单机多卡（Single Machine Multi-GPU）并行训练方法，重点探索 torch.distributed 和 torch.nn.parallel.DistributedDataParallel (DDP) 的使用。

随着深度学习模型规模的不断增长，单张 GPU 的计算能力往往难以满足高效训练的需求，因此，利用多张 GPU 进行数据并行加速训练成为一种常见的优化方案。

本实验采用 PyTorch 框架，实现一个基于 Transformer 结构的 NLP 模型，并使用 DDP 进行多 GPU 训练，以验证其性能和正确性。


## 2.实验目的
掌握 PyTorch 分布式训练框架的基本概念，包括 torch.distributed、DistributedDataParallel 等核心 API。

实现基于 DDP 的数据并行训练，在单机多 GPU 设备上进行深度学习模型训练，并理解其工作原理。

理解 DistributedSampler 在多 GPU 训练中的作用，确保数据在多个进程间均匀分配。


## 3.硬件要求

2张 GPU（4090、V100、A100等）或 CPU 。


## 4.技术原理

本实验的核心是 PyTorch 的 DistributedDataParallel (DDP)，它用于在多个 GPU  设备之间高效地并行化训练过程。其基本原理如下：

### 数据并行（Data Parallelism）

- 采用 单机多 GPU 数据并行策略，每张 GPU 运行一个独立的训练进程（采用 torch.multiprocessing 进行进程管理）。

- 训练数据集在多个 GPU 之间划分，每张 GPU 仅处理部分数据，并行计算梯度。

- DistributedSampler 确保每张 GPU 处理的数据不会重复，从而保证训练效率和正确性。

### DDP 训练流程

每个 GPU 进程独立计算梯度。

PyTorch DDP 通过 梯度同步（All-Reduce） 机制，在后向传播时将所有 GPU 的梯度聚合并同步，以确保模型参数在所有 GPU 上保持一致。

## 5.实验流程

### 环境配置



In [ ]:
!pip install torch

### 5.1. transformer模型定义

导入所需要的包，并定义了一个基本的 transformer 模型

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.distributed as dist
import torch.nn.functional as F
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import DistributedSampler
import math

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

class MultiHeadSelfAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadSelfAttention, self).__init__()
        assert d_model % num_heads == 0
        self.d_k = d_model // num_heads
        self.num_heads = num_heads
        
        self.q_linear = nn.Linear(d_model, d_model)
        self.k_linear = nn.Linear(d_model, d_model)
        self.v_linear = nn.Linear(d_model, d_model)
        self.out_linear = nn.Linear(d_model, d_model)
        
    def forward(self, x, mask=None):
        batch_size = x.shape[0]
        
        q = self.q_linear(x).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        k = self.k_linear(x).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        v = self.v_linear(x).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        
        scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(self.d_k)
        if mask is not None:
            scores = scores.masked_fill(mask == 0, float('-1e9'))
        
        attn_weights = F.softmax(scores, dim=-1)
        output = torch.matmul(attn_weights, v)
        
        output = output.transpose(1, 2).contiguous().view(batch_size, -1, self.num_heads * self.d_k)
        return self.out_linear(output)

class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(FeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
    
    def forward(self, x):
        return self.fc2(F.relu(self.fc1(x)))

class TransformerBlock(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super(TransformerBlock, self).__init__()
        self.attention = MultiHeadSelfAttention(d_model, num_heads)
        self.norm1 = nn.LayerNorm(d_model)
        self.ffn = FeedForward(d_model, d_ff)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x, mask=None):
        attn_out = self.attention(x, mask)
        x = self.norm1(x + self.dropout(attn_out))
        ffn_out = self.ffn(x)
        x = self.norm2(x + self.dropout(ffn_out))
        return x

class Transformer(nn.Module):
    def __init__(self, vocab_size, d_model=512, num_heads=8, d_ff=2048, num_layers=6, max_len=5000):
        super(Transformer, self).__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoding = PositionalEncoding(d_model, max_len)
        self.layers = nn.ModuleList([TransformerBlock(d_model, num_heads, d_ff) for _ in range(num_layers)])
        self.out_linear = nn.Linear(d_model, vocab_size)
    
    def forward(self, x, mask=None):
        x = self.embedding(x)
        x = self.pos_encoding(x)
        mask = torch.triu(torch.ones(x.shape[1], x.shape[1]), diagonal=1)
        for layer in self.layers:
            x = layer(x, mask)
        return self.out_linear(x)

model = Transformer(1000, 32, 8, 64, 2, 5000)

print(model)

### 5.2.数据集定义

该类定义了一个模拟数据集，其中第 i 条数据类似 [i, i, i, i, i, i, i]

在训练的过程中，我们希望模型能够捕捉到这条性质

In [ ]:
class NLPDataset(Dataset):
    def __init__(self, size, length):
        self.data = []
        for i in range(size):
            self.data.append(torch.full((length, ), i))

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]

dataset = NLPDataset(12, 10)
for data in dataset:
    print(data)

### 5.3训练核心代码实现

该段训练代码实现了主要的训练逻辑，包括数据集的使用，模型的实例化，DDP方法和具体的训练步骤

在 PyTorch 分布式训练中，每个训练进程称为一个“worker”，每个 worker 会处理自己分配的一部分数据和模型副本。

world_size：表示所有参与训练的进程总数。例如，如果你在 2 张 GPU 上各启动 1 个进程，world_size 就是 2。

rank：表示当前进程的编号，从 0 开始。例如，有两个进程的话，rank 可以是 0 或 1。每个进程通过 rank 来区分身份并处理属于自己的数据子集。

这两个参数在 dist.init_process_group() 中被用来初始化通信组，使得每个进程知道自己是谁以及整个训练的结构。

In [1]:
def train(rank, world_size):
    # 设置模型和训练的超参数
    VOCAB_SIZE = 100        # 词汇表大小
    D_MODEL = 12            # 嵌入维度 / 模型维度
    NUM_HEADS = 4           # 注意力头数量
    D_FF = 24               # 前馈层的中间维度
    NUM_LAYERS = 2          # Transformer 层数
    MAX_LEN = 100           # 输入序列的最大长度

    DATASET_SIZE = 12       # 样本数量（总数据条数）
    DATASET_LENGTH = 10     # 每条样本的长度
    BATCH_SIZE = 2          # 每个进程的 batch 大小

    # 创建自定义数据集
    dataset = NLPDataset(size=DATASET_SIZE, length=DATASET_LENGTH)

    # 为分布式训练设置采样器，每个 rank（进程）处理自己的一份数据
    sampler = DistributedSampler(dataset, num_replicas=world_size, rank=rank)

    # 使用带分布式采样器的数据加载器
    dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, sampler=sampler)

    # 设置后端：GPU 可用就使用 nccl，否则使用 gloo（CPU）
    backend = "nccl" if torch.cuda.is_available() else "gloo"
    device = torch.device(f"cuda:{rank}" if torch.cuda.is_available() else "cpu")

    # 初始化当前进程的分布式环境
    dist.init_process_group(backend=backend, rank=rank, world_size=world_size)

    # 实例化 Transformer 模型并移动到对应设备
    model = Transformer(
        vocab_size=VOCAB_SIZE,
        d_model=D_MODEL,
        num_heads=NUM_HEADS,
        d_ff=D_FF,
        num_layers=NUM_LAYERS,
        max_len=MAX_LEN,
    ).to(device)

    # 将模型封装为分布式模型（DDP）
    # DDP 会自动将模型 warp 成一个数据并行的形式，这样不同的进程在执行 forward 的时候，DDP 会自动进行数据并行的相关处理，例如反向传播的时候会自动同步梯度
    # DDP 的初始化需要传入整个进程通信组的相关信息，例如有多少个进程参与此次训练，每个进程的通信地址是什么，从而建立卡间通信网络
    # device_ids 对应当前进程使用的 GPU 编号，从而保证不同进程的模型放到不用的 GPU 上。
    # DistributedSampler 确保 DDP 模型在不同进程上获得的数据不同，反向传播以后，每个进程保存的模型梯度是不同的，这时根据之前建立的进程组信息，DDP 会自动将不同进程的模型梯度同步，同步以后结果等于执行一次 batch_size 为 DP 组数的训练。因此 global_batch_size = num_dp_group * batch_size_per_group
    model = DDP(model, device_ids=[rank] if torch.cuda.is_available() else None)

    # 使用交叉熵作为损失函数
    criterion = nn.CrossEntropyLoss()

    # 使用 SGD 作为优化器
    optimizer = optim.SGD(model.parameters(), lr=0.01)

    # 开始训练循环
    for epoch in range(200):
        for batch, data in enumerate(dataloader):
            # 目标是当前句子的下一位
            label = data[:, 1:].to(device)       # 标签是输入右移一位
            data = data[:, :-1].to(device)       # 输入去掉最后一个 token

            optimizer.zero_grad()                # 清空梯度
            outputs = model(data)                # 前向传播

            # 计算损失：输出和标签都 reshape 成二维，适配交叉熵
            loss = criterion(outputs.view(-1, VOCAB_SIZE), label.view(-1))

            loss.backward()                      # 反向传播
            optimizer.step()                     # 更新参数

            # 只在 rank==0 的主进程上打印日志，避免重复
            if rank == 0:
                print(f"Epoch {epoch}, Batch {batch}, Loss: {loss.item()}")

    # 训练完成后销毁分布式进程组
    dist.destroy_process_group()


### 5.4启动训练

假设你已经写好了训练脚本，例如叫做 train.py，可以通过如下命令启动分布式训练：

torchrun --nproc-per-node=2 train.py

参数解释：

torchrun：PyTorch 推荐的方式来启动多进程训练（取代老的 python -m torch.distributed.launch）。

--nproc-per-node=2：表示在本机上启动 2 个进程，分别对应 rank=0 和 rank=1。

In [ ]:
!bash run.sh